In [ ]:
import numpy as np
import pandas as pd
import seaborn as sb

In [ ]:
rw=pd.read_csv('/Users/valarian/SJTU/SJTU/毕业论文/Data/result/queue_simulation/2closed/const/1.5_1_1.5_3_rw.csv', index_col = 0)
rl1=pd.read_csv('/Users/valarian/SJTU/SJTU/毕业论文/Data/result/queue_simulation/2closed/const/1.5_1_1.5_3_rl1.csv', index_col = 0)
rld=pd.read_csv('/Users/valarian/SJTU/SJTU/毕业论文/Data/result/queue_simulation/2closed/const/1.5_1_1.5_3_rld.csv', index_col = 0)
rl2=pd.read_csv('/Users/valarian/SJTU/SJTU/毕业论文/Data/result/queue_simulation/2closed/const/1.5_1_1.5_3_rl2.csv', index_col = 0)

In [ ]:
rw1 = rw[rw.position == 1.0][7000:]
rw2 = rw[rw.position == 2.0][7000:]
rw1.info()
rw2.info()

In [ ]:
np.mean(rw1.arr.diff()[1:])

In [ ]:
np.mean(rw2.arr.diff()[1:])

In [ ]:
rw2.arr[:6000] - rw1.arr[:6000]

In [ ]:
np.random.seed(2)
a = np.random.exponential(1,2)
b = np.random.exponential(1,2)
a==b

In [ ]:
#G(M, N) calculation
def G(M, N, Lambda, Miu):
    '''
    M: number of servers in the network
    N: number of jobs in the network
    Lambda: vector of arrival rate at each server
    Miu: vector of service rate at each server
    '''
    
    if N == 0:
        return 1
    elif M == 1:
        return Lambda[0]**N / Miu[0]
    else:
        return Lambda[M-1]/Miu[M-1] * G(M, N-1, Lambda, Miu) + G(M-1, N, Lambda, Miu)
    
if __name__ == '__main__':
    np.random.seed(3)
    M = 4
    N = 10
    Lambda = [6, 6]
    Miu = [6, 8]
    %time G(M, N, Lambda, Miu)

In [ ]:
#closed network with two queues and exponential propagation delay
from tqdm import tqdm


T = 0
N = 50000
n_customer = 200
np.random.seed(1)
prop_delay1 = np.random.exponential(0.5, N)
prop_delay2 = np.random.exponential(0.5, N)
miu1 = 1/6
miu2 = 1/8

_ = np.random.exponential(miu1, N)
ins1 = [sum(_[:i+1]) for i in range(n_customer)]
ins2 = []
delay1 = []
delay2 = []
scheduler = [[s, 1] for s in ins1]

rw, W = pd.DataFrame(columns = ['place', 'arr', 'ser', 'lea']), n_customer
rl1, L1 = pd.DataFrame(columns=['n_sys', 't_sys']), 0
rld1, Ld1 = pd.DataFrame(columns=['n_sys', 't_sys']), 0
rl2, L2 = pd.DataFrame(columns=['n_sys', 't_sys']), 0
rld2, Ld2 = pd.DataFrame(columns=['n_sys', 't_sys']), 0

for i in range(n_customer):
    rw.loc[i] = [1, 0, sum(_[:i]), sum(_[:i])+_[i]]

for n in tqdm(range(N)):

    T, obj = scheduler[0]

    if obj == 1:
        rl1.loc[L1], L1 = [len(ins1), T], L1+1
        rld1.loc[Ld1], Ld1 = [len(delay1), T], Ld1+1
        rw.loc[W], W = [-1, T, T, T+prop_delay1[n]], W+1
        delay1.append(T+prop_delay1[n])
        scheduler.append([T+prop_delay1[n], -1])
        ins1.pop(0)
    elif obj == 2:
        rl2.loc[L2], L2 = [len(ins2), T], L2+1
        rld2.loc[Ld2], Ld2 = [len(delay2), T], Ld2+1
        rw.loc[W], W = [-2, T, T, T+prop_delay2[n]], W+1
        delay2.append(T+prop_delay2[n])
        scheduler.append([T+prop_delay2[n], -2])
        ins2.pop(0)
    elif obj == -1:
        rld1.loc[Ld1], Ld1 = [len(delay1), T], Ld1+1
        rl2.loc[L2], L2 = [len(ins2), T], L2+1
        if ins2: rw.loc[W], W = [2, T, ins2[-1], ins2[-1]+np.random.exponential(miu2)], W+1
        else: rw.loc[W], W = [2, T, T, T+np.random.exponential(miu2)], W+1
        ins2.append(rw.iloc[W-1][3])
        scheduler.append([rw.iloc[W-1][3], 2])
        delay1.pop(0)
    else:
        rld2.loc[Ld2], Ld2 = [len(delay2), T], Ld2+1
        rl1.loc[L1], L1 = [len(ins1), T], L1+1
        if ins1: rw.loc[W], W = [1, T, ins1[-1], ins1[-1]+np.random.exponential(miu1)], W+1
        else: rw.loc[W], W = [1, T, T, T+np.random.exponential(miu1)], W+1
        ins1.append(rw.iloc[W-1][3])
        scheduler.append([rw.iloc[W-1][3], 1])
        delay2.pop(0)
    scheduler.pop(0)
    scheduler.sort()
#print(rw)


In [ ]:
np.average(rw.loc[rw.place == 2.0].arr.diff()[10000:])

In [ ]:
prop_delay1

In [ ]:
#closed network with two queues and constant propagation delay
from tqdm import tqdm


T = 0
N = 20000
n_customer = 100
np.random.seed(1)
prop_delay1 = 0
prop_delay2 = 0
miu1 = 1/10
miu2 = 1/10

_ = np.random.exponential(miu1, N)
ins1 = [sum(_[:i+1]) for i in range(n_customer)]
ins2 = []
delay1 = []
delay2 = []
scheduler = [[s, 1] for s in ins1]

rw, W = pd.DataFrame(columns = ['place', 'arr', 'ser', 'lea']), n_customer
rl1, L1 = pd.DataFrame(columns=['n_sys', 't_sys']), 0
rld1, Ld1 = pd.DataFrame(columns=['n_sys', 't_sys']), 0
rl2, L2 = pd.DataFrame(columns=['n_sys', 't_sys']), 0
rld2, Ld2 = pd.DataFrame(columns=['n_sys', 't_sys']), 0

for i in range(n_customer):
    rw.loc[i] = [1, 0, sum(_[:i]), sum(_[:i])+_[i]]

for n in tqdm(range(N)):

    T, obj = scheduler[0]

    if obj == 1:
        rl1.loc[L1], L1 = [len(ins1), T], L1+1
        rld1.loc[Ld1], Ld1 = [len(delay1), T], Ld1+1
        rw.loc[W], W = [-1, T, T, T+prop_delay1], W+1
        delay1.append(T+prop_delay1)
        scheduler.append([T+prop_delay1, -1])
        ins1.pop(0)
    elif obj == 2:
        rl2.loc[L2], L2 = [len(ins2), T], L2+1
        rld2.loc[Ld2], Ld2 = [len(delay2), T], Ld2+1
        rw.loc[W], W = [-2, T, T, T+prop_delay2], W+1
        delay2.append(T+prop_delay2)
        scheduler.append([T+prop_delay2, -2])
        ins2.pop(0)
    elif obj == -1:
        rld1.loc[Ld1], Ld1 = [len(delay1), T], Ld1+1
        rl2.loc[L2], L2 = [len(ins2), T], L2+1
        if ins2: rw.loc[W], W = [2, T, ins2[-1], ins2[-1]+np.random.exponential(miu2)], W+1
        else: rw.loc[W], W = [2, T, T, T+np.random.exponential(miu2)], W+1
        ins2.append(rw.iloc[W-1][3])
        scheduler.append([rw.iloc[W-1][3], 2])
        delay1.pop(0)
    else:
        rld2.loc[Ld2], Ld2 = [len(delay2), T], Ld2+1
        rl1.loc[L1], L1 = [len(ins1), T], L1+1
        if ins1: rw.loc[W], W = [1, T, ins1[-1], ins1[-1]+np.random.exponential(miu1)], W+1
        else: rw.loc[W], W = [1, T, T, T+np.random.exponential(miu1)], W+1
        ins1.append(rw.iloc[W-1][3])
        scheduler.append([rw.iloc[W-1][3], 1])
        delay2.pop(0)
    scheduler.pop(0)
    scheduler.sort()
#print(rw)




In [ ]:
np.average(rw.loc[rw.place == 1.0].arr.diff()[10000:])

In [ ]:
test1 = rw

In [ ]:
np.average(test2.arr)

In [ ]:
np.average(rw.arr.diff()[1:])

In [ ]:
test1.loc[df]

In [ ]:
test2.info()

In [ ]:
sb.distplot(test1.loc[(test1.position == 2.0)&(test1.arr<50)].arr.diff()[1:])
sb.distplot(np.random.exponential(0.1, 20000))
sb.distplot(test2.loc[(test2.position == 2.0)&(test2.arr<50)].arr.diff()[1:])

In [ ]:
#closed network with two queues and propagation delay

T = 0
N = 20
cap = 5
np.random.seed(1)
prop_delay = 0
a = [0]*cap
s1 = np.random.exponential(1/10, N)
s2 = np.random.exponential(1/10, N)


ins1 = []
ins2 = []
rw, W = pd.DataFrame(columns = ['arrl', 'ser1', 'lea1', 'arr2', 'ser2', 'lea2']), 0
rl1, L1 = pd.DataFrame(columns=['n_sys', 't_sys']), 0
rq1, Q1 = pd.DataFrame(columns=['n_que', 't_que']), 0
rl2, L2 = pd.DataFrame(columns=['n_sys', 't_sys']), 0
rq2, Q2 = pd.DataFrame(columns=['n_que', 't_que']), 0

for n in range(N):
    T = a[0]
    if ins1:
        while ins1[0] <= T:
            rl1.loc[L1], L1 = [L, ins1[0]], L1+1
            rq1.loc[Q1], Q1 = [L-1, ins1[0]], Q1+1
            ins1.pop(0)
            if not ins1: break
    if ins2:
        while ins2[0][0] <= T:
            L = sum(pd.DataFrame(ins2).loc[:,1]<=T)
            rl2.loc[L2], L2 = [L, ins2[0][0]], L2+1
            rq2.loc[Q2], Q2 = [L-1, ins2[0][0]], Q2+1
            ins2.pop(0)
            if not ins2: break

            
    if ins1:
        if ins2:
            rw.loc[W], W = [T, ins1[-1], ins1[-1]+s1[n], ins1[-1]+s1[n]+prop_delay, max(ins1[-1]+s1[n]+prop_delay, ins2[-1][0]), 
                            max(ins1[-1]+s1[n]+prop_delay, ins2[-1][0])+s2[n]], W+1
            #rw
            while ins2[0][0] <= ins1[-1]+s1[n]+prop_delay:
                L = sum(pd.DataFrame(ins2).loc[:,1]<=ins1[-1]+s1[n]+prop_delay)
                rl2.loc[L2], L2 = [L, ins2[0][0]], L2+1
                rq2.loc[Q2], Q2 = [L-1, ins2[0][0]], Q2+1
                ins2.pop(0)
                if not ins2: break
            rl2.loc[L2], L2 = [len(ins2), ins1[-1]+s1[n]+prop_delay], L2+1
            rq2.loc[Q2], Q2 = [len(ins2)-1, ins1[-1]+s1[n]+prop_delay], Q2+1 
            if ins2: ins2.append([ins2[-1][0]+s2[n], ins1[-1]+s1[n]+prop_delay])
            else: ins2.append([ins1[-1]+s1[n]+prop_delay+s2[n], ins1[-1]+s1[n]+prop_delay])
            ins2.sort()
            
            a.append(max(ins1[-1]+s1[n]+prop_delay, ins2[-1][0])+s2[n])
        else: 
            rw.loc[W], W = [T, ins1[-1], ins1[-1]+s1[n], ins1[-1]+s1[n]+prop_delay, ins1[-1]+s1[n]+prop_delay, 
                            ins1[-1]+s1[n]+prop_delay+s2[n]], W+1
            #rw
            rl2.loc[L2], L2 = [0, ins1[-1]+s1[n]+prop_delay], L2+1
            ins2.append([T+s1[n]+prop_delay+s2[n], T+s1[n]+prop_delay])
            ins2.sort()
            
            a.append(ins1[-1]+s1[n]+prop_delay+s2[n])
            
        rl1.loc[L1], L1 = [len(ins1), T], L1+1
        rq1.loc[Q1], Q1 = [len(ins1)-1, T], Q1+1    
        ins1.append(ins1[-1]+s1[n])
        ins1.sort()
        
    else:
        if ins2:
            rw.loc[W], W = [T, T, T+s1[n], T+s1[n]+prop_delay, max(T+s1[n]+prop_delay, ins2[-1][0]), 
                            max(T+s1[n]+prop_delay, ins2[-1][0])+s2[n]], W+1
            #rw
            while ins2[0][0] <= T+s1[n]+prop_delay:
                L = sum(pd.DataFrame(ins2).loc[:,1]<=T+s1[n]+prop_delay)
                rl2.loc[L2], L2 = [L, ins2[0][0]], L2+1
                rq2.loc[Q2], Q2 = [L-1, ins2[0][0]], Q2+1
                ins2.pop(0)
                if not ins2: break
            rl2.loc[L2], L2 = [len(ins2), T+s1[n]+prop_delay], L2+1
            rq2.loc[Q2], Q2 = [len(ins2)-1, T+s1[n]+prop_delay], Q2+1 
            if ins2: ins2.append([ins2[-1][0]+s2[n], T+s1[n]+prop_delay])
            else: ins2.append([T+s1[n]+prop_delay+s2[n], T+s1[n]+prop_delay])
            ins2.sort()
            
            a.append(max(T+s1[n]+prop_delay, ins2[-1][0])+s2[n])
        else: 
            rw.loc[W], W = [T, T, T+s1[n], T+s1[n]+prop_delay, T+s1[n]+prop_delay, T+s1[n]+prop_delay+s2[n]], W+1
            #rw
            rl2.loc[L2], L2 = [0, T+s1[n]+prop_delay], L2+1
            ins2.append([T+s1[n]+prop_delay+s2[n], T+s1[n]+prop_delay])
            ins2.sort()
            
            a.append(T+s1[n]+prop_delay+s2[n])
    
        rl1.loc[L1], L1 = [0, T], L1+1
        ins1.append(T+s1[n])
        ins1.sort()
    
    a.pop(0)
    print('len: %d, %d, %d'%(len(a), len(ins1), len(ins2)))

In [ ]:
a = [[1,2], [2,3]]
max([])

In [ ]:
#closed network with two queues and propagation delay

T = 0
N = 20
cap = 5
np.random.seed(1)
prop_delay = 0
s1 = np.random.exponential(1/10, N)
s2 = np.random.exponential(1/10, N)

ins1 = [[sum(s1[:i+1]), 0, s1[i]] for i in range(cap)]
ins2 = []
delay = []
rw, W = pd.DataFrame(columns = ['position', 'arr2', 'ser2', 'lea2']), 0
rl1, L1 = pd.DataFrame(columns=['n_sys', 't_sys']), 0
rl2, L2 = pd.DataFrame(columns=['n_sys', 't_sys']), 0
rl3, L3 = pd.DataFrame(columns=['n_sys', 't_sys']), 0

for n in range(N):
    if not ins1:
        if not ins2:
            ins1.append([delay[0][0]+s1[n], delay[0][0], s1[n]])
            L = sum(pd.DataFrame(delay).loc[:,1]<delay[0][0])
            rl3.loc[L3], L3 = [L, delay[0][0]], L3+1
            rl2.loc[L2], L2 = [0, delay[0][0]], L2+1
            rl2.loc[L2], L2 = [1, delay[0][0]+s2[n]], L2+1
            rw2.loc[W2], W2 = [T+prop_delay, T+prop_delay, T+prop_delay+s2[n]], W2+1
        else:
            ins1.append([ins2[0][0]+s1[n], ins2[0][0], s1[n]])
            L = sum(pd.DataFrame(delay).loc[:,1]<ins2[0][0])
            rl2.loc[L2], L2 = [L, ins2[0][0]], L2+1
            rl2.loc[L2], L2 = [L-1, ins2[0][0]], L2+1
            ins2.pop(0)
        rl1.loc[L1], L1 = [0, ins1[0][1]], L1+1
            
    T = ins1[0][0]        
    rw1.loc[W1], W1 = [ins1[0][1], ins1[0][0]-ins1[0][2], ins1[0][0]], W+1
    
    if ins1:
        while ins1[0][0] < T:
            L = sum(pd.DataFrame(ins1).loc[:,1]<T)
            rl1.loc[L1], L1 = [L, ins1[0][0]], L1+1
            rq1.loc[Q1], Q1 = [L-1, ins1[0][0]], Q1+1
            rl3.loc[L3], L3 = [len(delay), ins1[0][0]]
            delay.append([ins1[0][0]+prop_delay, ins1[0][0]])
            delay.sort()
            ins1.pop(0)
            if not ins1: break
    if delay:
        while delay[0][0] < T:
            L = sum(pd.DataFrame(delay).loc[:,1]<T)
            rl3.loc[L3], L3 = [L, delay[0][0]], L3+1
            rl2.loc[L2], L2 = [len(ins2), delay[0][0]], L2+1
            if ins2: 
                ins2.append([ins2[-1][0]+s2[n], delay[0][0]])
                rq2.loc[Q2], Q2 = [len(ins2)-1, delay[0][0]], Q2+1
                rw2.loc[W2], W2= [T+prop_delay, ins2[-1][0], ins2[-1][0]+s2[n]], W2+1
            else: 
                ins2.append([delay[0][0]+s2[n], delay[0][0]])
                rw2.loc[W2], W2 = [T+prop_delay, T+prop_delay, T+prop_delay+s2[n]], W2+1
            ins2.sort()
            delay.pop(0)
            if not delay: break
    if ins2:
        while ins2[0][0] < T:
            L = sum(pd.DataFrame(ins2).loc[:,1]<T)
            rl2.loc[L2], L2 = [L, ins2[0][0]], L2+1
            rq2.loc[Q2], Q2 = [L-1, ins2[0][0]], Q2+1
            rl1.loc[L1], L1 = [len(ins1), ins2[0][0]], L1+1
            if ins1: 
                ins1.append([ins1[-1][0]+s1[n], ins2[0][0], s1[n+1]])
                rq1.loc[Q1], Q1 = [len(ins1)-1, ins2[0][0]], Q1+1
            else: 
                ins1.append([ins2[0][0]+s1[n], ins2[0][0], s1[n+1]])
                ins1.sort()
            ins2.pop(0)
            if not ins2: break
        
    
print('over')

In [ ]:
np.arange(0, 1, 0.1)

In [ ]:
# closed network with two queues and propagation delay
import numpy as np
import pandas as pd

# configuration = []
# for i in range(5,15):
#     for j in range(5, 15):
#         configuration.append([i*0.1, j*0.1])

configuration = [[0.5, 0.5]]

for setting in configuration:
    L = min(setting[0], setting[1])*0.9
    R = 1.1*max(setting[0], setting[1])
    for de in np.arange(L, R, (R-L)/2):
        T = 0
        N = 10000
        n_customer = 20
        np.random.seed(1)
        p_delay = np.random.exponential(de, N)
        miu1 = setting[0]
        miu2 = setting[1]
        
        result, R = pd.DataFrame(columns=['miu1', 'miu2', 'de', 'arr_avg_1', 'arr_avg_2']), 0

        _ = np.random.exponential(miu1, N)
        ins1 = [sum(_[:i + 1]) for i in range(n_customer)]
        ins2 = []
        delay = []
        scheduler = [[s, 1] for s in ins1]

        rw, W = pd.DataFrame(columns=['position', 'arr', 'ser', 'lea']), 5
        rl1, L1 = pd.DataFrame(columns=['n_sys', 't_sys']), 0
        rld, Ld = pd.DataFrame(columns=['n_sys', 't_sys']), 0
        rl2, L2 = pd.DataFrame(columns=['n_sys', 't_sys']), 0
        for i in range(5):
            rw.loc[i] = [1, 0, sum(_[:i]), sum(_[:i]) + _[i]]
        for n in range(N):

            prop_delay = p_delay[n]
            T, obj = scheduler[0]

            if obj == 1:
                rl1.loc[L1], L1 = [len(ins1), T], L1 + 1
                rld.loc[Ld], Ld = [len(delay), T], Ld + 1
                rw.loc[W], W = [0, T, T, T + prop_delay], W + 1
                delay.append(T + prop_delay)
                scheduler.append([T + prop_delay, 0])
                ins1.pop(0)
            elif obj == 2:
                rl2.loc[L2], L2 = [len(ins2), T], L2 + 1
                rl1.loc[L1], L1 = [len(ins1), T], L1 + 1
                if ins1:
                    rw.loc[W], W = [1, T, ins1[-1], ins1[-1] + np.random.exponential(miu1)], W + 1
                else:
                    rw.loc[W], W = [1, T, T, T + np.random.exponential(miu1)], W + 1
                ins1.append(rw.iloc[W - 1][3])
                scheduler.append([rw.iloc[W - 1][3], 1])
                ins2.pop(0)
            else:
                rld.loc[Ld], Ld = [len(delay), T], Ld + 1
                rl2.loc[L2], L2 = [len(ins2), T], L2 + 1
                if ins2:
                    rw.loc[W], W = [2, T, ins2[-1], ins2[-1] + np.random.exponential(miu2)], W + 1
                else:
                    rw.loc[W], W = [2, T, T, T + np.random.exponential(miu2)], W + 1
                ins2.append(rw.iloc[W - 1][3])
                scheduler.append([rw.iloc[W - 1][3], 2])
                delay.pop(0)
            scheduler.pop(0)
            scheduler.sort()

    result.loc[R], R = [setting[0], setting[1], de, np.average(rw[rw.position == 1.0].arr.diff()[1:]), np.average(rw[rw.position == 2.0].arr.diff()[1:])], R+1
    result.to_csv('testresult.csv')
    print('over')

In [ ]:
np.average(rw[rw.position == 1.0].arr.diff()[1:])

In [32]:
# closed network with two queues and 2 propagation delay processes

import numpy as np
import pandas as pd
from tqdm import tqdm

configuration = [[1.0, 1.0],
                 [2.0, 2.0],
                 [3.0, 3.0],
                 [1.0, 2.0],
                 [1.0, 3.0]]

re, index = pd.DataFrame(columns=['miu1', 'miu2', 'd1', 'd2', 'arr1', 'arr2', 'arrd1', 'arrd2', 'p10', 'p20', 'pd10',
                                 'pd20']), 0

for setting in tqdm(configuration):
    L, R = min(setting), max(setting)
    for de in [0.0, 1.0/L, 2.0/(L+R), 1.0/R, 10.0/R, 100.0/R]:
        for i in [1]:
            T = 0
            N = 40000
            n_customer = 100
            np.random.seed(1)
            if i:
                prop_delay1 = de
                prop_delay2 = 0

            miu1 = setting[0]
            miu2 = setting[1]

            _ = np.random.exponential(miu1, N)
            ins1 = [sum(_[:i+1]) for i in range(n_customer)]
            ins2 = []
            delay1 = []
            delay2 = []
            scheduler = [[s, 1] for s in ins1]

            rw, W = pd.DataFrame(columns = ['place', 'arr', 'ser', 'lea']), n_customer
            rl1, L1 = pd.DataFrame(columns=['n_sys', 't_sys']), 0
            rld1, Ld1 = pd.DataFrame(columns=['n_sys', 't_sys']), 0
            rl2, L2 = pd.DataFrame(columns=['n_sys', 't_sys']), 0
            rld2, Ld2 = pd.DataFrame(columns=['n_sys', 't_sys']), 0

            for i in range(n_customer):
                rw.loc[i] = [1, 0, sum(_[:i]), sum(_[:i])+_[i]]

            for n in range(N):

                T, obj = scheduler[0]

                if obj == 1:
                    rl1.loc[L1], L1 = [len(ins1), T], L1+1
                    rld1.loc[Ld1], Ld1 = [len(delay1), T], Ld1+1
                    rw.loc[W], W = [-1, T, T, T+prop_delay1], W+1
                    delay1.append(T+prop_delay1)
                    scheduler.append([T+prop_delay1, -1])
                    ins1.pop(0)
                elif obj == 2:
                    rl2.loc[L2], L2 = [len(ins2), T], L2+1
                    rld2.loc[Ld2], Ld2 = [len(delay2), T], Ld2+1
                    rw.loc[W], W = [-2, T, T, T+prop_delay2], W+1
                    delay2.append(T+prop_delay2)
                    scheduler.append([T+prop_delay2, -2])
                    ins2.pop(0)
                elif obj == -1:
                    rld1.loc[Ld1], Ld1 = [len(delay1), T], Ld1+1
                    rl2.loc[L2], L2 = [len(ins2), T], L2+1
                    if ins2: rw.loc[W], W = [2, T, ins2[-1], ins2[-1]+np.random.exponential(miu2)], W+1
                    else: rw.loc[W], W = [2, T, T, T+np.random.exponential(miu2)], W+1
                    ins2.append(rw.iloc[W-1][3])
                    scheduler.append([rw.iloc[W-1][3], 2])
                    delay1.pop(0)
                else:
                    rld2.loc[Ld2], Ld2 = [len(delay2), T], Ld2+1
                    rl1.loc[L1], L1 = [len(ins1), T], L1+1
                    if ins1: rw.loc[W], W = [1, T, ins1[-1], ins1[-1]+np.random.exponential(miu1)], W+1
                    else: rw.loc[W], W = [1, T, T, T+np.random.exponential(miu1)], W+1
                    ins1.append(rw.iloc[W-1][3])
                    scheduler.append([rw.iloc[W-1][3], 1])
                    delay2.pop(0)
                scheduler.pop(0)
                scheduler.sort()
            arr1 = np.average(rw[rw.place == 1.0].arr.diff()[1:])
            arr2 = np.average(rw[rw.place == 2.0].arr.diff()[1:])
            arrd1 = np.average(rw[rw.place == -1.0].arr.diff()[1:])
            arrd2 = np.average(rw[rw.place == -2.0].arr.diff()[1:])
            p10 = sum(rl1[rl1.n_sys == 0.0].t_sys.diff()[1:])/T
            p20 = sum(rl2[rl2.n_sys == 0.0].t_sys.diff()[1:])/T
            pd10 = sum(rld1[rld1.n_sys == 0.0].t_sys.diff()[1:])/T
            pd20 = sum(rld2[rld2.n_sys == 0.0].t_sys.diff()[1:])/T
            re.loc[index], index = [miu1, miu2, prop_delay1, prop_delay2, arr1, arr2, arrd1, arrd2, p10, p20, pd10, pd20], index+1
            del T, N, n_customer, _, ins1, ins2, delay1, delay2, scheduler, rw, W, rl1, L1, rld1, Ld1, rl2, L2, rld2, Ld2
            del miu1, miu2, prop_delay1, prop_delay2, arr1, arr2, arrd1, arrd2, p10, p20, pd10, pd20

re.to_csv('constantDelayResult2.csv')




  0%|                                                    | 0/5 [00:00<?, ?it/s]


 20%|████████                                | 1/5 [30:43<2:02:54, 1843.62s/it]


 40%|███████████████▏                      | 2/5 [1:01:36<1:32:19, 1846.48s/it]


 60%|██████████████████████▊               | 3/5 [1:32:11<1:01:25, 1842.81s/it]


 80%|████████████████████████████████        | 4/5 [2:02:55<30:43, 1843.23s/it]


100%|████████████████████████████████████████| 5/5 [2:33:37<00:00, 1842.97s/it]

In [31]:
# closed network with two queues and 2 propagation delay processes

import numpy as np
import pandas as pd
from tqdm import tqdm

configuration = [[1.0, 1.0]]

re, index = pd.DataFrame(columns=['miu1', 'miu2', 'd1', 'd2', 'arr1', 'arr2', 'arrd1', 'arrd2', 'p10', 'p20', 'pd10',
                                 'pd20']), 0

for setting in tqdm(configuration):
    L, R = min(setting), max(setting)
    for de in [0.0]:
        for i in [1]:
            T = 0
            N = 40000
            n_customer = 100
            np.random.seed(1)
            if i:
                prop_delay1 = de
                prop_delay2 = 0

            miu1 = setting[0]
            miu2 = setting[1]

            _ = np.random.exponential(miu1, N)
            ins1 = [sum(_[:i+1]) for i in range(n_customer)]
            ins2 = []
            delay1 = []
            delay2 = []
            scheduler = [[s, 1] for s in ins1]

            rw, W = pd.DataFrame(columns = ['place', 'arr', 'ser', 'lea']), n_customer
            rl1, L1 = pd.DataFrame(columns=['n_sys', 't_sys']), 0
            rld1, Ld1 = pd.DataFrame(columns=['n_sys', 't_sys']), 0
            rl2, L2 = pd.DataFrame(columns=['n_sys', 't_sys']), 0
            rld2, Ld2 = pd.DataFrame(columns=['n_sys', 't_sys']), 0

            for i in range(n_customer):
                rw.loc[i] = [1, 0, sum(_[:i]), sum(_[:i])+_[i]]

            for n in range(N):

                T, obj = scheduler[0]

                if obj == 1:
                    rl1.loc[L1], L1 = [len(ins1), T], L1+1
                    rld1.loc[Ld1], Ld1 = [len(delay1), T], Ld1+1
                    rw.loc[W], W = [-1, T, T, T+prop_delay1], W+1
                    delay1.append(T+prop_delay1)
                    scheduler.append([T+prop_delay1, -1])
                    ins1.pop(0)
                elif obj == 2:
                    rl2.loc[L2], L2 = [len(ins2), T], L2+1
                    rld2.loc[Ld2], Ld2 = [len(delay2), T], Ld2+1
                    rw.loc[W], W = [-2, T, T, T+prop_delay2], W+1
                    delay2.append(T+prop_delay2)
                    scheduler.append([T+prop_delay2, -2])
                    ins2.pop(0)
                elif obj == -1:
                    rld1.loc[Ld1], Ld1 = [len(delay1), T], Ld1+1
                    rl2.loc[L2], L2 = [len(ins2), T], L2+1
                    if ins2: rw.loc[W], W = [2, T, ins2[-1], ins2[-1]+np.random.exponential(miu2)], W+1
                    else: rw.loc[W], W = [2, T, T, T+np.random.exponential(miu2)], W+1
                    ins2.append(rw.iloc[W-1][3])
                    scheduler.append([rw.iloc[W-1][3], 2])
                    delay1.pop(0)
                else:
                    rld2.loc[Ld2], Ld2 = [len(delay2), T], Ld2+1
                    rl1.loc[L1], L1 = [len(ins1), T], L1+1
                    if ins1: rw.loc[W], W = [1, T, ins1[-1], ins1[-1]+np.random.exponential(miu1)], W+1
                    else: rw.loc[W], W = [1, T, T, T+np.random.exponential(miu1)], W+1
                    ins1.append(rw.iloc[W-1][3])
                    scheduler.append([rw.iloc[W-1][3], 1])
                    delay2.pop(0)
                scheduler.pop(0)
                scheduler.sort()
                
            arr1 = np.average(rw[rw.place == 1.0].arr.diff()[1:])
            arr2 = np.average(rw[rw.place == 2.0].arr.diff()[1:])
            arrd1 = np.average(rw[rw.place == -1.0].arr.diff()[1:])
            arrd2 = np.average(rw[rw.place == -2.0].arr.diff()[1:])
            p10 = sum(rl1[rl1.n_sys == 0.0].t_sys.diff()[1:])/T
            p20 = sum(rl2[rl2.n_sys == 0.0].t_sys.diff()[1:])/T
            pd10 = sum(rld1[rld1.n_sys == 0.0].t_sys.diff()[1:])/T
            pd20 = sum(rld2[rld2.n_sys == 0.0].t_sys.diff()[1:])/T
            re.loc[index], index = [miu1, miu2, prop_delay1, prop_delay2, arr1, arr2, arrd1, arrd2, p10, p20, pd10, pd20], index+1
            del T, N, n_customer, _, ins1, ins2, delay1, delay2, scheduler
            del miu1, miu2, prop_delay1, prop_delay2, arr1, arr2, arrd1, arrd2, p10, p20, pd10, pd20






  0%|                                                    | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [30]:
re

,miu1,miu2,d1,d2,arr1,arr2,arrd1,arrd2,p10,p20,pd10,pd20
0,1.0,1.0,0.0,0.0,1.003886,1.006032,1.006032,1.013699,0.867185,0.440607,0.999946,0.999738


In [7]:
re

,miu1,miu2,d1,d2,arr1,arr2,arrd1,arrd2,p10,p20,pd10,pd20
0,1.0,1.0,0.000000,0.0,0.0,0.000000,0.509144,0.500779,0.021150,0.003894,0.999946,0.999738
1,1.0,1.0,1.000000,0.0,0.0,0.320168,0.350882,0.337978,0.002589,0.023444,0.367148,0.999527
2,1.0,1.0,1.000000,0.0,0.0,0.320168,0.350882,0.337978,0.002589,0.023444,0.367148,0.999527
3,1.0,1.0,1.000000,0.0,0.0,0.320168,0.350882,0.337978,0.002589,0.023444,0.367148,0.999527
4,1.0,1.0,10.000000,0.0,0.0,0.337355,0.336488,0.329775,0.010115,0.004155,0.000000,0.998741
5,1.0,1.0,100.000000,0.0,0.0,0.395997,0.377225,0.361046,0.112802,0.118096,0.000000,0.990962
6,2.0,2.0,0.000000,0.0,0.0,0.000000,1.018288,1.001557,0.021150,0.003894,0.999946,0.999738
7,2.0,2.0,0.500000,0.0,0.0,0.351093,0.819811,0.843502,0.017195,0.002964,0.780985,0.999712
8,2.0,2.0,0.500000,0.0,0.0,0.351093,0.819811,0.843502,0.017195,0.002964,0.780985,0.999712
9,2.0,2.0,0.500000,0.0,0.0,0.351093,0.819811,0.843502,0.017195,0.002964,0.780985,0.999712
